In [ ]:
!pip install tensorflow gradio --quiet

import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import gradio as gr
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load trained model
MODEL_PATH = "/content/drive/MyDrive/models/plantvillage1_model.h5"
model = load_model(MODEL_PATH)
print(f"Model loaded. Input shape: {model.input_shape}")

# Model input size
IMG_HEIGHT, IMG_WIDTH = 96, 96

# Class labels
CLASS_LABELS = [
    'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust',
    'Apple___healthy', 'Blueberry___healthy', 'Cherry___Powdery_mildew',
    'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
    'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy',
    'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)',
    'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot',
    'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight',
    'Potato___healthy', 'Raspberry___healthy', 'Strawberry___Leaf_scorch',
    'Strawberry___healthy', 'Squash___Powdery_mildew',
    'Soybean___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight',
    'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
]

# Prediction function with image overlay
def predict_leaf_disease(img):
    try:
        if img is None:
            return None, "No image uploaded"

        # Convert to RGB
        img = img.convert('RGB')

        # Maintain aspect ratio
        img.thumbnail((IMG_WIDTH, IMG_HEIGHT), Image.ANTIALIAS)
        new_img = Image.new("RGB", (IMG_WIDTH, IMG_HEIGHT), (0,0,0))
        new_img.paste(img, ((IMG_WIDTH - img.width)//2, (IMG_HEIGHT - img.height)//2))
        img_resized = new_img

        # Preprocess
        img_array = np.array(img_resized)/255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        preds = model.predict(img_array)[0]
        pred_index = np.argmax(preds)
        pred_label = CLASS_LABELS[pred_index]
        confidence = preds[pred_index] * 100

        # Extract plant and disease
        if 'healthy' in pred_label.lower():
            parts = pred_label.split('___')
            plant_name = parts[0].capitalize()
            disease_name = "Healthy"
            health_percent = round(confidence, 2)
        else:
            parts = pred_label.split('___')
            plant_name = parts[0].capitalize()
            disease_name = parts[1].replace('_', ' ').capitalize()
            health_percent = round(100 - confidence, 2)

        # Overlay text on the image
        img_display = img_resized.resize((256, 256))  # enlarge for better view
        draw = ImageDraw.Draw(img_display)
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 16)
        except:
            font = ImageFont.load_default()
        text = f"Plant: {plant_name}\nDisease: {disease_name}\nHealth: {health_percent}%"
        draw.rectangle([0, 0, 256, 60], fill=(0,0,0,127))
        draw.text((5, 5), text, fill="white", font=font)

        return img_display, f"Prediction done"

    except Exception as e:
        print("Error:", e)
        return None, str(e)

# Gradio Interface
iface = gr.Interface(
    fn=predict_leaf_disease,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="pil", label="Leaf Image"), gr.Textbox(label="Status")],
    title="🌿 Plant Disease Detection",
    description="Upload a leaf image to predict the plant, disease, and health %. The result will be shown on the image.",
    theme="default",
    examples=[
        ["/content/drive/MyDrive/test_images/tomato_leaf1.jpg"],
        ["/content/drive/MyDrive/test_images/apple_leaf1.jpg"]
    ]
)

iface.launch()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model loaded. Input shape: (None, 96, 96, 3)
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ede798acbf9b659aca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
